In [1]:
import pandas as pd
import os

# create an empty DataFrame with columns
df = pd.DataFrame(columns=['state', 'attractive place', 'date', 'review', 'type', 'likes'])

In [2]:
# print the resulting DataFrame
print(df)

Empty DataFrame
Columns: [state, attractive place, date, review, type, likes]
Index: []


In [3]:
# Set the directory where the folders containing the CSV files are located
folder_directory = 'C:\\Users\\vivek\\Downloads\\tourist places data (1) (3)'
# Get a list of all the folders in the directory
folders = os.listdir(folder_directory)

# Create an empty list to hold the dataframes
df_list = []

In [4]:
# Loop through each folder and read in the CSV files as dataframes
for folder in folders:
    # Get a list of all the CSV files in the folder
    csv_files = [f for f in os.listdir(os.path.join(folder_directory, folder)) if f.endswith('.csv')]
    
    # Create an empty list to hold the dataframes for this folder
    folder_df_list = []
    
    
    # create an empty DataFrame with columns
    df5 = pd.DataFrame(columns=['state', 'attractive place', 'date', 'review', 'type', 'likes'])
    # Loop through each CSV file and read it into a dataframe
    for csv_file in csv_files:
        csv_path = os.path.join(folder_directory, folder, csv_file)
        attraction_df5 = pd.read_csv(csv_path, encoding='ISO-8859-1')
        attraction_name5 = csv_file.split('.')[0]  # get the attraction name from the filename
        state_name5 = os.path.basename(folder)  # set the state name as appropriate
        # add new columns for the state name and attraction name
        attraction_df5['state'] = state_name5
        attraction_df5['attractive place'] = attraction_name5
        # reorder the columns so that the new columns are in the correct order
        attraction_df5 = attraction_df5[['state', 'attractive place', 'date', 'review', 'type', 'likes']]
        df5=pd.concat([df5, attraction_df5], ignore_index=True)
        folder_df_list.append(df5)
    
    # Concatenate all the dataframes for this folder into a single dataframe
    folder_df = pd.concat(folder_df_list, ignore_index=True)
    
    # Append the folder dataframe to the list of dataframes
    df_list.append(folder_df)

# Concatenate all the dataframes in the list into a single dataframe
df_final = pd.concat(df_list, ignore_index=True)

In [5]:
df_final

,state,attractive place,date,review,type,likes
0,AndhraPradesh,Borra Caves,20-03-2023,A must visit place in Vizag. Lot of ups and do...,Friends,2
1,AndhraPradesh,Borra Caves,03-03-2023,A good place to visit. Just be careful of the ...,Couples,7
2,AndhraPradesh,Borra Caves,13-01-2023,Completely blanked over by the terrific territ...,Friends,6
3,AndhraPradesh,Borra Caves,07-01-2023,"Exceptional place, an absolute must visit for...",Couples,1
4,AndhraPradesh,Borra Caves,04-01-2023,The Bora caves is awsome. Liked it too much. T...,Couples,4
...,...,...,...,...,...,...
46881,Uttarakhand,Kedarnath,22-06-2020,Heaven on earth. Easily reach by bus from hari...,Solo,1
46882,Uttarakhand,Kedarnath,06-11-2022,"Being very genuine, after seeing the temple an...",Friends,7
46883,Uttarakhand,Kedarnath,31-08-2020,Very energetic power of God . Just close your ...,Friends,6
46884,Uttarakhand,Kedarnath,23-02-2022,"Baba Kedarnath ki Jai. Holy site, awesome weat...",Couples,3


In [6]:
!pip install vaderSentiment

In [6]:
df_final.dropna(subset=['review'], inplace=True)

In [7]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [8]:
stopwords = set(stopwords.words("english"))

In [9]:
def clean_reviews(df):
    df = df.astype(str)
    # Remove punctuation
    df = df.apply(lambda x: re.sub(r'[^\w\s]', '', x))
    # Convert to lowercase
    df = df.apply(lambda x: x.lower())
    # Remove numbers
    df = df.apply(lambda x: re.sub(r'\d', '', x))
    # Remove extra whitespaces
    df = df.apply(lambda x: re.sub(r'\s+', ' ', x))
    # Remove stopwords
    df = df.apply(lambda x: " ".join([word for word in x.split() if word not in stopwords]))
    return df

In [10]:
# Initialize the sentiment analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

# Define a function to get the sentiment score for a review
def get_sentiment_score(review):
    score = sentiment_analyzer.polarity_scores(review)["compound"]
    return (score + 1) * 2.5  # Scale the score to 10

In [11]:
def filter_reviews(reviews, aspects):
    filtered_reviews = []
    for review in reviews:
        for aspect in aspects:
            if aspect in review.lower():
                filtered_reviews.append(review)
                break
    return filtered_reviews

In [12]:
df_final["Cleaned Reviews"]= clean_reviews(df_final["review"])

In [13]:
Food = ["meal", "dish", "cuisine", "cooking", "food", "beverage", "drink", "snack", "appetizer", "dessert",   
                "breakfast", "brunch", "lunch", "dinner", "supper", "mealtime", "feast", "banquet",   
                "sandwich", "burger", "pizza", "taco", "wrap", "deli", "fast food", "street food",  
                "soup", "stew", "chowder", "bisque", "broth", "gravy",    "salad", "coleslaw", "pasta salad", 
                "fruit salad", "greek salad", "caesar salad",    "vegetable", "fruit", "produce", "legume", 
                "root vegetable", "leafy green", "garden fresh",    "meat", "poultry", "beef", "pork", "lamb",
                 "venison", "bacon", "sausage", "ribs",    "fish", "seafood", "shellfish", "tuna", "salmon", 
                 "cod", "shrimp", "crab", "lobster",    "spice", "herb", "seasoning", "salt", "pepper", 
                 "garlic", "ginger", "cilantro", "basil",    "dairy", "milk", "cheese", "yogurt", "cream", 
                 "butter", "egg", "custard",    "sweet", "sugar", "candy", "chocolate", "cookie", "cake", 
                 "pie", "pastry", "doughnut",    "bake", "roast", "grill", "fry", "saute", "boil", "steam", 
                 "microwave", "cook",    "biryani", "butter chicken", "chaat", "chana masala", "chole", 
                 "dosa", "idli", "jalebi", "kebab",    "naan", "paneer tikka", "paratha", "rasgulla", 
                 "samosa", "tandoori chicken", "tikka masala", "vindaloo"]

In [14]:
Transport = ["road", "vehicle", "car", "truck", "van", "suv", "sedan", "road trip", "ride", "drive",    
                            "bike", "cycling", "mountain biking", "road cycling", "bmx", "touring", "ride", "pedal",    
                            "flight", "airplane", "aeroplane", "jet", "passenger plane", "commercial flight", "charter flight", 
                            "air travel", "soar",    "train", "railway", "locomotive", "passenger train", "freight train", 
                            "subway", "metro", "commute", "rail",    "bus", "coach", "shuttle", "public transportation", 
                            "school bus", "city bus", "tour bus", "ride", "bus stop",    "ship", "vessel", "cruise", "cargo ship", 
                            "tanker", "sailboat", "sea travel", "maritime",    "ferry", "water taxi", "passenger ferry", "car ferry",
                            "catamaran", "hovercraft", "ferry terminal", "boat ride",    "car", "automobile", "motor vehicle", "roadster",
                            "convertible", "sports car", "sedan", "road trip", "drive",    "motorcycle", "bike", "harley davidson",
                            "dirt bike", "touring bike", "sport bike", "motorcycle ride", "bike lane",    
                            "walking", "hiking", "trekking", "strolling", "pedestrian", "foot travel", "walking trail", "sidewalk"]

In [15]:
Hotel = ["accommodation", "lodging", "inn", "motel", "resort", "hostel", "bed and breakfast",
                "guesthouse",    "vacation rental", "apartment", "suite", "room", "double room", 
                "single room", "triple room",    "quadruple room", "king room", "queen room",
                "twin room", "executive room", "deluxe room", "superior room",    "standard room", 
                "budget room", "luxury room", "ocean view room", "city view room", "pool view room",   
                "non-smoking room", "smoking room", "pet-friendly room", "accessibility room", 
                "bathroom", "shower",    "bathtub", "toilet", "sink", "bidet", "linens", "towels", 
                "pillows", "blankets", "air conditioning",    "heating", "fan", "television", "cable",
                "internet", "wifi", "telephone", "mini-bar", "refrigerator",    "microwave", 
                "coffee maker", "room service", "concierge", "front desk", "reception", "check-in",
                 "check-out",    "early check-in", "late check-out", "key card", "room key", "security",
                  "lock", "alarm", "fire alarm",    "smoke detector", "emergency exits", "hotel amenities", 
                    "restaurant", "bar", "lounge", "pool", "gym",    "spa", "business center", "meeting room",
                    "conference room", "banquet room", "parking", "valet parking",    "car rental", "tour desk",
                    "luggage storage", "currency exchange", "ATM", "gift shop", "newspaper",    "laundry service", 
                    "dry cleaning", "ironing service", "beauty salon", "health club", "jacuzzi", "sauna",    
                    "steam room", "massage", "yoga", "fitness center", "game room", "kids club", "library", 
                    "garden",    "terrace", "sun deck", "beach", "beach club", "beachfront", "oceanfront", 
                    "ocean view", "mountain view",    "city view", "river view", "hotel location", "nearby attractions",
                    "nearby restaurants", "nearby shopping",    "nearby transportation", "central location", 
                    "downtown location", "airport shuttle", "taxi", "public transportation",    "metro", 
                     "train station", "bus station", "highway", "freeway", "expressway", "hotel chain",   
                    "hotel brand", "hotel rating", "hotel review", "hotel recommendation", "hotel booking", 
                    "hotel reservation",    "hotel deal", "hotel discount", "hotel coupon", "hotel package", 
                    "hotel promotion", "hotel offer",    "hotel gift card", "hotel loyalty program", "hotel rewards",
                    "hotel membership", "hotel club"]

In [21]:
import nltk
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

In [44]:
# split each review into a list of words
reviews_list = [review.split() for review in df_final["Cleaned Reviews"]]

# create the Word2Vec model
model = Word2Vec(sentences=reviews_list, vector_size=100, window=5, min_count=1, workers=4)

# get the word embeddings for each review
reviews_embeddings = np.array([np.mean([model.wv[word] for word in review], axis=0) for review in reviews_list])


In [60]:
reviews_embeddings

array([[ 0.22063744,  0.1962265 ,  0.4286354 , ...,  0.15669867,
        -0.4222952 ,  0.4499093 ],
       [ 0.61335903,  0.49160674,  0.42436808, ..., -0.42574695,
        -0.84698504,  0.822694  ],
       [ 0.3860441 ,  0.21656863,  0.32899475, ..., -0.4939091 ,
         0.12315945,  0.02830079],
       ...,
       [ 0.53984076, -0.42913973,  1.0702435 , ...,  0.40699205,
         0.11738481, -0.1651762 ],
       [-0.01347504,  0.07984886,  0.51144034, ..., -0.2084336 ,
         0.27284527,  0.3793813 ],
       [-0.2478983 , -0.4784478 ,  0.85860497, ..., -0.14239864,
         0.20689921,  0.05264013]], dtype=float32)

In [45]:
# set the number of clusters to 3 (for food, transport, and hotel reviews)
num_clusters = 3

# create the KMeans model and fit it to the reviews embeddings
kmeans_model = KMeans(n_clusters=num_clusters, random_state=1)
kmeans_model.fit(reviews_embeddings)

# get the cluster labels for each review
cluster_labels = kmeans_model.labels_

In [61]:
df_final["cluster_label"] = cluster_labels

# divide the reviews into food, transport, and hotel reviews based on the cluster labels
df_final["Food Reviews"] = df_final["cluster_label"] == 0
df_final["Transport Reviews"] = df_final["cluster_label"] == 1
df_final["Hotel Reviews"] = df_final["cluster_label"] == 2


In [17]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [18]:
# Calculate the TF-IDF scores for each word in the reviews
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(df_final['review'])

# Cluster the reviews using KMeans
kmeans = KMeans(n_clusters=3).fit(tfidf)
df_final['cluster'] = kmeans.labels_

# Assign each cluster to its corresponding category
food_cluster = df_final[df_final['cluster'] == 0]
transport_cluster = df_final[df_final['cluster'] == 1]
hotel_cluster = df_final[df_final['cluster'] == 2]

In [19]:
food_cluster

,state,attractive place,date,review,type,likes,Cleaned Reviews,cluster
0,AndhraPradesh,Borra Caves,20-03-2023,A must visit place in Vizag. Lot of ups and do...,Friends,2,must visit place vizag lot ups downs tend get ...,0
2,AndhraPradesh,Borra Caves,13-01-2023,Completely blanked over by the terrific territ...,Friends,6,completely blanked terrific territory awesome ...,0
4,AndhraPradesh,Borra Caves,04-01-2023,The Bora caves is awsome. Liked it too much. T...,Couples,4,bora caves awsome liked much lighting system g...,0
7,AndhraPradesh,Borra Caves,26-10-2022,An unusual natural formation which is exciting...,Friends,2,unusual natural formation exciting look walk t...,0
8,AndhraPradesh,Borra Caves,02-08-2022,Not a place to visit at 60 plusToo strenuous p...,Solo,4,place visit plustoo strenuous physicallygroans...,0
...,...,...,...,...,...,...,...,...
46880,Uttarakhand,Kedarnath,21-04-2021,- up can find inner peace â®ï¸\n \n - long j...,Friends,7,find inner peace âï long journey u never feel ...,0
46881,Uttarakhand,Kedarnath,22-06-2020,Heaven on earth. Easily reach by bus from hari...,Solo,1,heaven earth easily reach bus haridwar till so...,0
46883,Uttarakhand,Kedarnath,31-08-2020,Very energetic power of God . Just close your ...,Friends,6,energetic power god close eyes feel,0
46884,Uttarakhand,Kedarnath,23-02-2022,"Baba Kedarnath ki Jai. Holy site, awesome weat...",Couples,3,baba kedarnath ki jai holy site awesome weathe...,0


In [66]:
df_final["review"][3]

"Exceptional place,  an absolute must visit for everyone. The caves run deep and long and are fairly friendly to climb up or down. The stalagmites are beautiful and certain openings with natural light piercing through makes it a photographers delight. For those afraid of dark places , the caves are amply lit bit alas with multi colored lights that change color. Despite the depth and length of the caves and presence of bats , the caves don't smell. There's ample air and one wouldn't feel claustrophobic unless someone is very sensitive. The caves were clean as well.Ticket pricing is not high ( 80 rupees for am adult).However It's sad and unfortunate  to see how we can spoil this natural Marvel by unruly behaviour. People go berserk with mindless howling, shouting, whistling and screaming. It's a shame people how people can be so insensitive about other tourist.Some points which i think the government needs to work on1) these are prehistoric caves offering raw infiltrred natural beauty, a

In [63]:
df_final

,state,attractive place,date,review,type,likes,Cleaned Reviews,cluster_label,Food Reviews,Transport Reviews,Hotel Reviews
0,AndhraPradesh,Borra Caves,20-03-2023,A must visit place in Vizag. Lot of ups and do...,Friends,2,must visit place vizag lot ups downs tend get ...,1,False,True,False
1,AndhraPradesh,Borra Caves,03-03-2023,A good place to visit. Just be careful of the ...,Couples,7,good place visit careful monkeys heavily packe...,2,False,False,True
2,AndhraPradesh,Borra Caves,13-01-2023,Completely blanked over by the terrific territ...,Friends,6,completely blanked terrific territory awesome ...,2,False,False,True
3,AndhraPradesh,Borra Caves,07-01-2023,"Exceptional place, an absolute must visit for...",Couples,1,exceptional place absolute must visit everyone...,2,False,False,True
4,AndhraPradesh,Borra Caves,04-01-2023,The Bora caves is awsome. Liked it too much. T...,Couples,4,bora caves awsome liked much lighting system g...,2,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
46881,Uttarakhand,Kedarnath,22-06-2020,Heaven on earth. Easily reach by bus from hari...,Solo,1,heaven earth easily reach bus haridwar till so...,2,False,False,True
46882,Uttarakhand,Kedarnath,06-11-2022,"Being very genuine, after seeing the temple an...",Friends,7,genuine seeing temple view around thats heaven...,0,True,False,False
46883,Uttarakhand,Kedarnath,31-08-2020,Very energetic power of God . Just close your ...,Friends,6,energetic power god close eyes feel,0,True,False,False
46884,Uttarakhand,Kedarnath,23-02-2022,"Baba Kedarnath ki Jai. Holy site, awesome weat...",Couples,3,baba kedarnath ki jai holy site awesome weathe...,0,True,False,False


In [58]:
df_final["Cleaned Reviews"][55]

'awesome place one go nearer nature beautiful lights make caves look beautiful must visit'

In [54]:
transport_reviews

,state,attractive place,date,review,type,likes,Cleaned Reviews,cluster_label
0,AndhraPradesh,Borra Caves,20-03-2023,A must visit place in Vizag. Lot of ups and do...,Friends,2,must visit place vizag lot ups downs tend get ...,1
9,AndhraPradesh,Borra Caves,25-07-2022,Beautiful place...must visit this place atleas...,Couples,4,beautiful placemust visit place atleast vizagu...,1
48,AndhraPradesh,Borra Caves,25-12-2019,Surely one of India's great sites it does not ...,Solo,6,surely one indias great sites disappoint get c...,1
55,AndhraPradesh,Borra Caves,05-12-2019,Awesome place and one can go nearer to nature....,Couples,5,awesome place one go nearer nature beautiful l...,1
56,AndhraPradesh,Borra Caves,23-11-2019,Borra caves is one of good place we can enjoy ...,Friends,7,borra caves one good place enjoy lot place tra...,1
...,...,...,...,...,...,...,...,...
46860,Uttarakhand,Kedarnath,20-02-2022,The place is amazing ....feel like you are in ...,Friends,4,place amazing feel like heaven,1
46866,Uttarakhand,Kedarnath,04-11-2022,"What an experience, after a long trek when you...",Friends,6,experience long trek reach top wont feel tired...,1
46869,Uttarakhand,Kedarnath,20-04-2021,The pure place full of peace and spirituality ...,Solo,3,pure place full peace spirituality must visit ...,1
46876,Uttarakhand,Kedarnath,22-02-2022,Its Just Heaven In India Can not say in words ...,Couples,4,heaven india say words awesome mind blowing place,1


In [ ]:
# Print the reviews in each category
for category, reviews in classified_reviews.items():
    print(category)
    print(reviews)


In [16]:
def food_reviews(df):
    food_reviews = filter_reviews(df['Cleaned Reviews'], Food)
    df["Food Reviews"]=pd.Series(food_reviews)
    df["Food Reviews"]=df["Food Reviews"].astype(str)
    df["Food scores"]=df["Food Reviews"].apply(get_sentiment_score)
    return df["Food scores"]

In [17]:
def transport_reviews(df):
    transport_reviews = filter_reviews(df['Cleaned Reviews'], Transport)
    df["Transport Reviews"]=pd.Series(transport_reviews)
    df["Transport Reviews"]=df["Transport Reviews"].astype(str)
    df["Transport Scores"]=df["Transport Reviews"].apply(get_sentiment_score)
    return df["Transport Scores"]

In [18]:
def hotel_reviews(df):
    hotel_reviews = filter_reviews(df["Cleaned Reviews"], Hotel)
    df["Hotel Reviews"]=pd.Series(hotel_reviews)
    df["Hotel Reviews"]=df["Hotel Reviews"].astype(str)
    df["Hotel Scores"]=df["Hotel Reviews"].apply(get_sentiment_score)
    return df["Hotel Scores"]

In [19]:
# Perform some calculation or operation on the CSV DataFrame
df.dropna(subset=["review"], inplace=True)

In [20]:
df_final["Cleaned Reviews"]= clean_reviews(df_final["review"])
df_final["Sentiment scores"] = df_final["Cleaned Reviews"].apply(get_sentiment_score)

In [21]:
df_final

,state,attractive place,date,review,type,likes,Cleaned Reviews,Sentiment scores
0,AndhraPradesh,Borra Caves,20-03-2023,A must visit place in Vizag. Lot of ups and do...,Friends,2,must visit place vizag lot ups downs tend get ...,2.62900
1,AndhraPradesh,Borra Caves,03-03-2023,A good place to visit. Just be careful of the ...,Couples,7,good place visit careful monkeys heavily packe...,4.58975
2,AndhraPradesh,Borra Caves,13-01-2023,Completely blanked over by the terrific territ...,Friends,6,completely blanked terrific territory awesome ...,4.86875
3,AndhraPradesh,Borra Caves,07-01-2023,"Exceptional place, an absolute must visit for...",Couples,1,exceptional place absolute must visit everyone...,4.40300
4,AndhraPradesh,Borra Caves,04-01-2023,The Bora caves is awsome. Liked it too much. T...,Couples,4,bora caves awsome liked much lighting system g...,4.72125
...,...,...,...,...,...,...,...,...
46881,Uttarakhand,Kedarnath,22-06-2020,Heaven on earth. Easily reach by bus from hari...,Solo,1,heaven earth easily reach bus haridwar till so...,4.56775
46882,Uttarakhand,Kedarnath,06-11-2022,"Being very genuine, after seeing the temple an...",Friends,7,genuine seeing temple view around thats heaven...,3.77650
46883,Uttarakhand,Kedarnath,31-08-2020,Very energetic power of God . Just close your ...,Friends,6,energetic power god close eyes feel,4.03100
46884,Uttarakhand,Kedarnath,23-02-2022,"Baba Kedarnath ki Jai. Holy site, awesome weat...",Couples,3,baba kedarnath ki jai holy site awesome weathe...,4.78400


In [22]:
food_reviews(df_final)

0        4.40300
1        4.72125
2        4.88275
3        4.54400
4        4.82825
          ...   
46881    2.50000
46882    2.50000
46883    2.50000
46884    2.50000
46885    2.50000
Name: Food scores, Length: 46886, dtype: float64

In [23]:
transport_reviews(df_final)

0        4.58975
1        4.86875
2        4.40300
3        4.69475
4        2.50000
          ...   
46881    2.50000
46882    2.50000
46883    2.50000
46884    2.50000
46885    2.50000
Name: Transport Scores, Length: 46886, dtype: float64

In [24]:
hotel_reviews(df_final)

0        4.40300
1        4.72125
2        4.88275
3        4.54400
4        4.97275
          ...   
46881    2.50000
46882    2.50000
46883    2.50000
46884    2.50000
46885    2.50000
Name: Hotel Scores, Length: 46886, dtype: float64

In [25]:
df_final

,state,attractive place,date,review,type,likes,Cleaned Reviews,Sentiment scores,Food Reviews,Food scores,Transport Reviews,Transport Scores,Hotel Reviews,Hotel Scores
0,AndhraPradesh,Borra Caves,20-03-2023,A must visit place in Vizag. Lot of ups and do...,Friends,2,must visit place vizag lot ups downs tend get ...,2.62900,exceptional place absolute must visit everyone...,4.40300,good place visit careful monkeys heavily packe...,4.58975,exceptional place absolute must visit everyone...,4.40300
1,AndhraPradesh,Borra Caves,03-03-2023,A good place to visit. Just be careful of the ...,Couples,7,good place visit careful monkeys heavily packe...,4.58975,bora caves awsome liked much lighting system g...,4.72125,completely blanked terrific territory awesome ...,4.86875,bora caves awsome liked much lighting system g...,4.72125
2,AndhraPradesh,Borra Caves,13-01-2023,Completely blanked over by the terrific territ...,Friends,6,completely blanked terrific territory awesome ...,4.86875,biggest cave india well maintained spots also ...,4.88275,exceptional place absolute must visit everyone...,4.40300,biggest cave india well maintained spots also ...,4.88275
3,AndhraPradesh,Borra Caves,07-01-2023,"Exceptional place, an absolute must visit for...",Couples,1,exceptional place absolute must visit everyone...,4.40300,place locted houra drive vizagour cab took rs ...,4.54400,millions year old good day spend nice driving ...,4.69475,place locted houra drive vizagour cab took rs ...,4.54400
4,AndhraPradesh,Borra Caves,04-01-2023,The Bora caves is awsome. Liked it too much. T...,Couples,4,bora caves awsome liked much lighting system g...,4.72125,cave unique gift nature cave deep nearly steps...,4.82825,place visit plustoo strenuous physicallygroans...,2.50000,get train vizagtrain vizag reach borra caves a...,4.97275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881,Uttarakhand,Kedarnath,22-06-2020,Heaven on earth. Easily reach by bus from hari...,Solo,1,heaven earth easily reach bus haridwar till so...,4.56775,nan,2.50000,nan,2.50000,nan,2.50000
46882,Uttarakhand,Kedarnath,06-11-2022,"Being very genuine, after seeing the temple an...",Friends,7,genuine seeing temple view around thats heaven...,3.77650,nan,2.50000,nan,2.50000,nan,2.50000
46883,Uttarakhand,Kedarnath,31-08-2020,Very energetic power of God . Just close your ...,Friends,6,energetic power god close eyes feel,4.03100,nan,2.50000,nan,2.50000,nan,2.50000
46884,Uttarakhand,Kedarnath,23-02-2022,"Baba Kedarnath ki Jai. Holy site, awesome weat...",Couples,3,baba kedarnath ki jai holy site awesome weathe...,4.78400,nan,2.50000,nan,2.50000,nan,2.50000


In [26]:
# Remove NaN values from the review column
df_final.dropna(subset=['review'], inplace=True)

In [27]:
df_final.head()

,state,attractive place,date,review,type,likes,Cleaned Reviews,Sentiment scores,Food Reviews,Food scores,Transport Reviews,Transport Scores,Hotel Reviews,Hotel Scores
0,AndhraPradesh,Borra Caves,20-03-2023,A must visit place in Vizag. Lot of ups and do...,Friends,2,must visit place vizag lot ups downs tend get ...,2.62900,exceptional place absolute must visit everyone...,4.40300,good place visit careful monkeys heavily packe...,4.58975,exceptional place absolute must visit everyone...,4.40300
1,AndhraPradesh,Borra Caves,03-03-2023,A good place to visit. Just be careful of the ...,Couples,7,good place visit careful monkeys heavily packe...,4.58975,bora caves awsome liked much lighting system g...,4.72125,completely blanked terrific territory awesome ...,4.86875,bora caves awsome liked much lighting system g...,4.72125
2,AndhraPradesh,Borra Caves,13-01-2023,Completely blanked over by the terrific territ...,Friends,6,completely blanked terrific territory awesome ...,4.86875,biggest cave india well maintained spots also ...,4.88275,exceptional place absolute must visit everyone...,4.40300,biggest cave india well maintained spots also ...,4.88275
3,AndhraPradesh,Borra Caves,07-01-2023,"Exceptional place, an absolute must visit for...",Couples,1,exceptional place absolute must visit everyone...,4.40300,place locted houra drive vizagour cab took rs ...,4.54400,millions year old good day spend nice driving ...,4.69475,place locted houra drive vizagour cab took rs ...,4.54400
4,AndhraPradesh,Borra Caves,04-01-2023,The Bora caves is awsome. Liked it too much. T...,Couples,4,bora caves awsome liked much lighting system g...,4.72125,cave unique gift nature cave deep nearly steps...,4.82825,place visit plustoo strenuous physicallygroans...,2.50000,get train vizagtrain vizag reach borra caves a...,4.97275


In [28]:
df_final.to_excel('AllData.xlsx', index=False)